<a href="https://colab.research.google.com/github/FanisBaygildin/Toronto-KSI-Risk-Bot/blob/main/Google_Maps_Route.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
import polyline
import pygeohash as pgh
from typing import List, Tuple, Dict

"""google_maps_route.py

Мини‑утилиты для:
    1. Geocoding: postal code (Toronto) → lat, lon
    2. Directions: получить до N альтернативных маршрутов
       и перевести их в список geohash5.

Ключ Google берётся из переменной среды ``GOOGLE_KEY``.
"""

GOOGLE_KEY = os.getenv("GOOGLE_KEY")
if not GOOGLE_KEY:
    raise RuntimeError("GOOGLE_KEY environment variable not set")

GEOCODE_URL = "https://maps.googleapis.com/maps/api/geocode/json"
DIRECTIONS_URL = "https://maps.googleapis.com/maps/api/directions/json"


def geocode_postal(code: str) -> Tuple[float, float]:
    """Return (lat, lon) for a Canadian postal code."""
    params = {
        "address": code,
        "components": "country:CA",
        "key": GOOGLE_KEY,
    }
    js = requests.get(GEOCODE_URL, params=params, timeout=10).json()
    if js.get("status") != "OK":
        raise ValueError(f"Geocoding error for '{code}': {js.get('status')}")
    loc = js["results"][0]["geometry"]["location"]
    return loc["lat"], loc["lng"]


def _decode(poly: str) -> List[Tuple[float, float]]:
    return polyline.decode(poly, precision=5)


def _uniq_geohash5(points: List[Tuple[float, float]]) -> List[str]:
    seen, out = set(), []
    for lat, lon in points:
        gh = pgh.encode(lat, lon, precision=5)
        if gh not in seen:
            seen.add(gh)
            out.append(gh)
    return out


def get_routes(origin_postal: str, dest_postal: str, max_alt: int = 3) -> List[Dict]:
    """Return list of route dicts with distance, duration, points, geohash5."""
    o_lat, o_lon = geocode_postal(origin_postal)
    d_lat, d_lon = geocode_postal(dest_postal)

    params = {
        "origin": f"{o_lat},{o_lon}",
        "destination": f"{d_lat},{d_lon}",
        "alternatives": "true" if max_alt > 1 else "false",
        "units": "metric",
        "mode": "driving",
        "key": GOOGLE_KEY,
    }
    js = requests.get(DIRECTIONS_URL, params=params, timeout=15).json()
    if js.get("status") != "OK":
        raise RuntimeError(f"Directions API: {js.get('status')}")

    routes = []
    for r in js.get("routes", [])[:max_alt]:
        leg = r["legs"][0]
        dist = leg["distance"]["value"]    # metres
        dur  = leg["duration"]["value"]    # seconds
        pts  = _decode(r["overview_polyline"]["points"])
        gh5  = _uniq_geohash5(pts)
        routes.append({
            "distance_m": dist,
            "duration_s": dur,
            "points": pts,
            "geohash5": gh5,
        })
    return routes


if __name__ == "__main__":
    import sys
    if len(sys.argv) != 3:
        print("Usage: python google_maps_route.py <origin_postal> <dest_postal>")
        sys.exit(1)
    for i, rt in enumerate(get_routes(sys.argv[1], sys.argv[2]), 1):
        km = rt['distance_m'] / 1000
        mins = rt['duration_s'] / 60
        print(f"Route {i}: {km:.1f} km, {mins:.0f} min, {len(rt['geohash5'])} cells")
